In [1]:
# --- INSTALL DEPENDENCIES ---
!pip install torch torchvision matplotlib opencv-python

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
from PIL import Image
import time
import copy

In [4]:
from google.colab import drive
drive.mount('/content/drive')

# SET YOUR DATA PATH
DATA_DIR = "/content/drive/MyDrive/Data"   # <--- YOUR FOLDER

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import os
import shutil

base = "/content/dataset"  # temp dataset structure
os.makedirs(base, exist_ok=True)

train_dir = f"{base}/train"
val_dir = f"{base}/val"

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# create classes
classes = ["Fire", "Non_Fire"]
for c in classes:
    os.makedirs(f"{train_dir}/{c}", exist_ok=True)
    os.makedirs(f"{val_dir}/{c}", exist_ok=True)

# copy train images
src_train_fire = f"{DATA_DIR}/Train_Data/Fire"
src_train_no = f"{DATA_DIR}/Train_Data/Non_Fire"

# copy test images
src_test_fire = f"{DATA_DIR}/Test_Data/Fire"
src_test_no = f"{DATA_DIR}/Test_Data/Non_Fire"

def copy_files(src, dst):
    for file in os.listdir(src):
        shutil.copy(os.path.join(src, file), dst)

# Fill train
copy_files(src_train_fire, f"{train_dir}/Fire")
copy_files(src_train_no, f"{train_dir}/Non_Fire")

# Fill validation
copy_files(src_test_fire, f"{val_dir}/Fire")
copy_files(src_test_no, f"{val_dir}/Non_Fire")

print("Dataset structured successfully for ImageFolder!")

Dataset structured successfully for ImageFolder!


In [16]:
import os
import shutil

DATA_DIR = "/content/drive/MyDrive/Data"
OUTPUT_DIR = "/content/dataset"

valid_exts = (".jpg", ".jpeg", ".png", ".bmp", ".webp")

def copy_images(src, dst):
    os.makedirs(dst, exist_ok=True)
    for f in os.listdir(src):
        if f.lower().endswith(valid_exts):
            shutil.copy(os.path.join(src, f), dst)

# Remove old dataset completely
shutil.rmtree(OUTPUT_DIR, ignore_errors=True)

# Recreate structure clean
os.makedirs(f"{OUTPUT_DIR}/train/Fire", exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/train/Non_Fire", exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/val/Fire", exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/val/Non_Fire", exist_ok=True)

# Copy images from Drive again (fresh copy)
copy_images(f"{DATA_DIR}/Train_Data/Fire", f"{OUTPUT_DIR}/train/Fire")
copy_images(f"{DATA_DIR}/Train_Data/Non_Fire", f"{OUTPUT_DIR}/train/Non_Fire")
copy_images(f"{DATA_DIR}/Test_Data/Fire", f"{OUTPUT_DIR}/val/Fire")
copy_images(f"{DATA_DIR}/Test_Data/Non_Fire", f"{OUTPUT_DIR}/val/Non_Fire")

print("Dataset copied fresh!")

Dataset copied fresh!


In [17]:
from PIL import Image
import os

def remove_corrupted(folder_path):
    removed = 0
    for filename in os.listdir(folder_path):
        path = os.path.join(folder_path, filename)
        try:
            with Image.open(path) as img:
                img.verify()
        except:
            os.remove(path)
            removed += 1
    return removed

folders = [
    "/content/dataset/train/Fire",
    "/content/dataset/train/Non_Fire",
    "/content/dataset/val/Fire",
    "/content/dataset/val/Non_Fire"
]

total_removed = sum(remove_corrupted(f) for f in folders)
print("Corrupted removed:", total_removed)


Corrupted removed: 389


In [18]:
import os

print("Train_Data/Fire:", len(os.listdir("/content/drive/MyDrive/Data/Train_Data/Fire")))
print("Train_Data/Non_Fire:", len(os.listdir("/content/drive/MyDrive/Data/Train_Data/Non_Fire")))

print("Test_Data/Fire:", len(os.listdir("/content/drive/MyDrive/Data/Test_Data/Fire")))
print("Test_Data/Non_Fire:", len(os.listdir("/content/drive/MyDrive/Data/Test_Data/Non_Fire")))

Train_Data/Fire: 2500
Train_Data/Non_Fire: 2500
Test_Data/Fire: 25
Test_Data/Non_Fire: 25


In [19]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

train_tf = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor()
])

val_tf = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

train_data = datasets.ImageFolder("/content/dataset/train", transform=train_tf)
val_data   = datasets.ImageFolder("/content/dataset/val", transform=val_tf)

train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_data, batch_size=16)

print("Classes:", train_data.classes)
print("Train samples:", len(train_data))
print("Val samples:", len(val_data))

Classes: ['Fire', 'Non_Fire']
Train samples: 4611
Val samples: 50


In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)

for p in model.parameters():
    p.requires_grad = False

model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, 2)  # Fire, Non_Fire
)

model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=1e-4)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 198MB/s]


In [21]:
def train(model, epochs=10):
    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")
        model.train()

        total, correct = 0, 0
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)

            optimizer.zero_grad()
            out = model(imgs)
            loss = criterion(out, labels)
            loss.backward()
            optimizer.step()

            _, pred = torch.max(out, 1)
            correct += (pred == labels).sum().item()
            total += labels.size(0)

        print("Train Acc:", correct / total)

        # validation
        model.eval()
        total, correct = 0, 0
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
                out = model(imgs)
                _, pred = torch.max(out, 1)
                correct += (pred == labels).sum().item()
                total += labels.size(0)

        print("Val Acc:", correct / total)

train(model, epochs=3)


Epoch 1/3
Train Acc: 0.9787464758186945
Val Acc: 0.98

Epoch 2/3
Train Acc: 0.9763608761656908
Val Acc: 0.98

Epoch 3/3
Train Acc: 0.9774452396443288
Val Acc: 0.96


In [22]:
MODEL_PATH = "/content/drive/MyDrive/fire_classifier_resnet50.pth"

torch.save(model.state_dict(), MODEL_PATH)

print("Model saved to:", MODEL_PATH)

Model saved to: /content/drive/MyDrive/fire_classifier_resnet50.pth


In [34]:
from google.colab import files

files.download("/content/drive/MyDrive/fire_classifier_resnet50.pth")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Load ResNet50
model_pred = models.resnet50(weights=None)
model_pred.fc = nn.Sequential(
    nn.Linear(model_pred.fc.in_features, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, 2)   # Fire, Non_Fire
)

model_pred.load_state_dict(torch.load("/content/drive/MyDrive/fire_classifier_resnet50.pth", map_location=DEVICE))
model_pred.to(DEVICE)
model_pred.eval()

print("Model loaded successfully!")

Model loaded successfully!


In [24]:
def predict_image(image_path):
    transform_pred = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor()
    ])

    img = Image.open(image_path).convert("RGB")
    img_t = transform_pred(img).unsqueeze(0).to(DEVICE)

    with torch.no_grad():
        out = model_pred(img_t)
        _, pred = torch.max(out, 1)

    classes = ["Fire", "Non_Fire"]
    return classes[pred.item()]

In [33]:
test_path = "/content/NF_3.jpg"   # example
print("Prediction:", predict_image(test_path))

Prediction: Non_Fire
